In [38]:
import pandas as pd

# Load CSVs
rover_df = pd.read_csv("data/rover.csv")
shelter_df = pd.read_csv("data/shelter.csv")
xhs_df = pd.read_csv("data/xhs.csv")

# Combine
combined_df = pd.concat([rover_df, shelter_df, xhs_df], ignore_index=True)

# Ensure numeric types
combined_df["Price"] = pd.to_numeric(combined_df["Price"], errors="coerce")
combined_df["Duration"] = pd.to_numeric(combined_df["Duration"], errors="coerce")
combined_df["Payment Received"] = pd.to_numeric(combined_df["Payment Received"], errors="coerce")

# Recalculate Daily Price & Tips, rounded to 2 decimals
combined_df["daily price"] = (combined_df["Price"] / combined_df["Duration"]).round(2)
combined_df["tips"] = (combined_df["Payment Received"] - combined_df["Price"]).clip(lower=0).round(2)

# Preview
combined_df


,Platform,Type,Name,Breed,Sex,Species,Spray/Neuter,Arrival Date,Departure Date,Duration,Price,Payment Received,Family Members,Notes,daily price,tips,months_active
0,Rover,Drop In,Sophie,Domestic Short,Female,Cat,Yes,22-Jun-2024,23-Jun-2024,2.0,42.0,42.0,NaN,NaN,21.00,0.0,NaN
1,Rover,Drop In,Delia,Domestic Short,Female,Cat,Yes,29-Jun-2024,1-Jul-2024,4.0,140.0,140.0,NaN,NaN,35.00,0.0,NaN
2,Rover,Drop In,Moomoo,Domestic Short,Male,Cat,Yes,1-Jul-2024,5-Jul-2024,3.0,64.0,64.0,NaN,NaN,21.33,0.0,NaN
3,Rover,Drop In,Sophie,Domestic Short,Female,Cat,Yes,4-Jul-2024,8-Jul-2024,5.0,150.0,150.0,NaN,NaN,30.00,0.0,NaN
4,Rover,Boarding,Migo,Chihuahua,Male,Dog,Yes,9-Jul-2024,10-Jul-2024,1.0,44.0,44.0,Draco/Indigo,NaN,44.00,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,xhs,Boarding,Uni,Shiba Inu,Male,Dog,yes,20/07/2025,24/07/2025,5.0,225.0,225.0,NaN,NaN,45.00,0.0,NaN
268,xhs,Boarding,Kylie,Aussie Doodle,Female,Dog,no,13/07/2025,23/07/2025,10.0,350.0,350.0,NaN,35一天,35.00,0.0,NaN
269,xhs,Boarding,雪蛤,Husky,Female,Dog,yes,05/08/2025,28/08/2025,22.0,450.0,450.0,NaN,NaN,20.45,0.0,NaN
270,xhs,Boarding,椰椰,Bichon Frise,Female,Dog,yes,19/08/2025,24/08/2025,5.0,150.0,150.0,NaN,NaN,30.00,0.0,NaN


In [39]:
# 1. Clean weird chars
for col in ["Arrival Date", "Departure Date"]:
    combined_df[col] = (
        combined_df[col]
        .astype(str)
        .str.strip()
        .str.replace(r"[^0-9A-Za-z/-]", "", regex=True)  # keep only numbers, letters, -, /
    )

# 2. Parse dates robustly
def parse_mixed_date(x):
    # Try DD/MM/YYYY or DD-MMM-YYYY first
    d = pd.to_datetime(x, dayfirst=True, errors="coerce", infer_datetime_format=True)
    if pd.isna(d):
        # fallback: try ISO or US-style
        d = pd.to_datetime(x, errors="coerce", infer_datetime_format=True)
    return d

combined_df["Arrival Date"] = combined_df["Arrival Date"].apply(parse_mixed_date)
combined_df["Departure Date"] = combined_df["Departure Date"].apply(parse_mixed_date)
combined_df

/var/folders/nl/h3_58vqx22x50_0gx2f0hqbm0000gn/T/ipykernel_30622/798924853.py:13: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  d = pd.to_datetime(x, dayfirst=True, errors="coerce", infer_datetime_format=True)


,Platform,Type,Name,Breed,Sex,Species,Spray/Neuter,Arrival Date,Departure Date,Duration,Price,Payment Received,Family Members,Notes,daily price,tips,months_active
0,Rover,Drop In,Sophie,Domestic Short,Female,Cat,Yes,2024-06-22,2024-06-23,2.0,42.0,42.0,NaN,NaN,21.00,0.0,NaN
1,Rover,Drop In,Delia,Domestic Short,Female,Cat,Yes,2024-06-29,2024-07-01,4.0,140.0,140.0,NaN,NaN,35.00,0.0,NaN
2,Rover,Drop In,Moomoo,Domestic Short,Male,Cat,Yes,2024-07-01,2024-07-05,3.0,64.0,64.0,NaN,NaN,21.33,0.0,NaN
3,Rover,Drop In,Sophie,Domestic Short,Female,Cat,Yes,2024-07-04,2024-07-08,5.0,150.0,150.0,NaN,NaN,30.00,0.0,NaN
4,Rover,Boarding,Migo,Chihuahua,Male,Dog,Yes,2024-07-09,2024-07-10,1.0,44.0,44.0,Draco/Indigo,NaN,44.00,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,xhs,Boarding,Uni,Shiba Inu,Male,Dog,yes,2025-07-20,2025-07-24,5.0,225.0,225.0,NaN,NaN,45.00,0.0,NaN
268,xhs,Boarding,Kylie,Aussie Doodle,Female,Dog,no,2025-07-13,2025-07-23,10.0,350.0,350.0,NaN,35一天,35.00,0.0,NaN
269,xhs,Boarding,雪蛤,Husky,Female,Dog,yes,2025-08-05,2025-08-28,22.0,450.0,450.0,NaN,NaN,20.45,0.0,NaN
270,xhs,Boarding,椰椰,Bichon Frise,Female,Dog,yes,2025-08-19,2025-08-24,5.0,150.0,150.0,NaN,NaN,30.00,0.0,NaN


In [40]:
def get_months_active(row):
    if pd.isna(row["Arrival Date"]) or pd.isna(row["Departure Date"]):
        return ""
    
    # Generate first day of each month between arrival and departure
    months = pd.date_range(row["Arrival Date"], row["Departure Date"], freq="MS").tolist()
    
    # Ensure both arrival and departure months are included
    months.append(pd.to_datetime(row["Arrival Date"]).replace(day=1))
    months.append(pd.to_datetime(row["Departure Date"]).replace(day=1))
    
    # Deduplicate + sort
    months = sorted(set(months))
    
    # Format YYYY-MM
    return ", ".join(m.strftime("%m/%Y") for m in months)

combined_df["months_active"] = combined_df.apply(get_months_active, axis=1)

# Check
combined_df

,Platform,Type,Name,Breed,Sex,Species,Spray/Neuter,Arrival Date,Departure Date,Duration,Price,Payment Received,Family Members,Notes,daily price,tips,months_active
0,Rover,Drop In,Sophie,Domestic Short,Female,Cat,Yes,2024-06-22,2024-06-23,2.0,42.0,42.0,NaN,NaN,21.00,0.0,06/2024
1,Rover,Drop In,Delia,Domestic Short,Female,Cat,Yes,2024-06-29,2024-07-01,4.0,140.0,140.0,NaN,NaN,35.00,0.0,"06/2024, 07/2024"
2,Rover,Drop In,Moomoo,Domestic Short,Male,Cat,Yes,2024-07-01,2024-07-05,3.0,64.0,64.0,NaN,NaN,21.33,0.0,07/2024
3,Rover,Drop In,Sophie,Domestic Short,Female,Cat,Yes,2024-07-04,2024-07-08,5.0,150.0,150.0,NaN,NaN,30.00,0.0,07/2024
4,Rover,Boarding,Migo,Chihuahua,Male,Dog,Yes,2024-07-09,2024-07-10,1.0,44.0,44.0,Draco/Indigo,NaN,44.00,0.0,07/2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,xhs,Boarding,Uni,Shiba Inu,Male,Dog,yes,2025-07-20,2025-07-24,5.0,225.0,225.0,NaN,NaN,45.00,0.0,07/2025
268,xhs,Boarding,Kylie,Aussie Doodle,Female,Dog,no,2025-07-13,2025-07-23,10.0,350.0,350.0,NaN,35一天,35.00,0.0,07/2025
269,xhs,Boarding,雪蛤,Husky,Female,Dog,yes,2025-08-05,2025-08-28,22.0,450.0,450.0,NaN,NaN,20.45,0.0,08/2025
270,xhs,Boarding,椰椰,Bichon Frise,Female,Dog,yes,2025-08-19,2025-08-24,5.0,150.0,150.0,NaN,NaN,30.00,0.0,08/2025


In [41]:
# Sort by Arrival Date (oldest → newest)
combined_df = combined_df.sort_values(by="Arrival Date", ascending=False).reset_index(drop=True)
# Save to CSV
output_path = "data/combined.csv"
combined_df.to_csv(output_path, index=False)

print(f"✅ Combined dataframe saved to {output_path}")
combined_df


✅ Combined dataframe saved to data/combined.csv


,Platform,Type,Name,Breed,Sex,Species,Spray/Neuter,Arrival Date,Departure Date,Duration,Price,Payment Received,Family Members,Notes,daily price,tips,months_active
0,Rover,Boarding,Murphy,Chihuahua,Female,Dog,Yes,2025-12-30,2025-01-02,3.0,228.0,183.0,NaN,NaN,76.00,0.0,"01/2025, 12/2025"
1,xhs,Boarding,Oliver,Dachshund Long,Male,Dog,yes,2025-09-18,2025-09-21,3.0,90.0,0.0,NaN,NaN,30.00,0.0,09/2025
2,xhs,Boarding,Oliver,Dachshund Long,Male,Dog,yes,2025-08-30,2025-09-05,6.0,180.0,0.0,NaN,NaN,30.00,0.0,"08/2025, 09/2025"
3,xhs,Boarding,Brownie,Dachshund Long,Male,Dog,no,2025-08-22,2025-09-13,22.0,566.0,0.0,Archie/Luna,一天一共3只80,25.73,0.0,"08/2025, 09/2025"
4,,Boarding,Luna,Australian Shepherd,Female,Dog,yes,2025-08-22,2025-09-13,22.0,566.0,0.0,Brownie/Archie,一天一共3只80,25.73,0.0,"08/2025, 09/2025"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,xhs,Boarding,福来,Domestic Short,Female,Cat,yes,2024-06-22,2024-07-01,10.0,172.6,172.6,麻薯/小七,NaN,17.26,0.0,"06/2024, 07/2024"
268,xhs,Boarding,小七,Domestic Short,Male,Cat,yes,2024-06-22,2024-07-01,10.0,172.6,172.6,福来/麻薯,NaN,17.26,0.0,"06/2024, 07/2024"
269,Rover,Drop In,Sophie,Domestic Short,Female,Cat,Yes,2024-06-22,2024-06-23,2.0,42.0,42.0,NaN,NaN,21.00,0.0,06/2024
270,xhs,Boarding,Appa,Maltese Mixed,Male,Dog,no,2024-06-16,2024-06-21,6.0,210.0,210.0,NaN,一天25,35.00,0.0,06/2024


In [43]:
import pandas as pd
import os

# Create the data folder if it doesn't exist
os.makedirs("data", exist_ok=True)

# Sample data
data = {
    "category": ["Others", "Transport", "Entertainment", "Grocery", "Pet","Subscription"],
    "date": pd.to_datetime(["2025-08-01", "2025-08-02", "2025-08-03", "2025-08-04", "2025-08-05", "2025-08-05"]),
    "amount": [15.5, 7.0, 20.0, 12.0, 8.5, 9.9]
}

# Create DataFrame
df = pd.DataFrame(data)

# Save to CSV in the data folder
file_path = "data/expenses.csv"
df.to_csv(file_path, index=False)

print(f"DataFrame saved to {file_path}")


DataFrame saved to data/expenses.csv
